In [54]:
import os

import numpy as np
from dotenv import load_dotenv
from plotly.figure_factory._ternary_contour import (
    _compute_grid,
    _contour_trace,
    _prepare_barycentric_coord,
    _ternary_layout,
)
from plotly.graph_objs import graph_objs as go
from pycalphad import Database, calculate

from chemical_charts.config import DOTENV_PATH

In [55]:
load_dotenv(DOTENV_PATH)

True

In this notebook I explore a more low level approach for creating a ternary contour line plot. This approach is more flexible and allows for more customization. It is dependent on private functions in the `plotly.figure_factory._ternary_contour` module. Essentially I am reimplementing the logic in the `create_ternary_contour` factory function in the aforementioned module.

## Setup (Do not include in class)

In [56]:
db_fname = "Al-Cu-Y.tdb"
db_path = os.path.join(os.getenv("TDB_DIR", ""), db_fname)
db_al_cu_y = Database(db_path)

In [57]:
comps = ['AL', 'CU', 'Y', 'VA']

In [58]:
c = calculate(db_al_cu_y, comps, 'FCC_A1', output='HM_MIX', T=830, P=101325, pdens=5000)

In [59]:
xs = c.X.values[0, 0, 0, :, 0]  # 1D array of Al compositions
ys = c.X.values[0, 0, 0, :, 1]  # 1D array of Cu compositions
zs = c.HM_MIX.values[0, 0, 0, :]  # 1D array of mixing enthalpies at these compositions

## Parameters the class needs to be initialized with

In [60]:
colorscale = "Bluered"
linecolor = "black"
coloring = "lines"
coordinates, values = np.array([xs, ys, 1 - xs - ys]), zs
showscale = False
ncontours = 5
pole_labels = ["Al", "Cu", "Y"]
width = 500
height = 500
title = "Al-Cu-Y"

## Logic to create the plot

In [61]:
barycentric_coordinates = _prepare_barycentric_coord(coordinates)
v_min, v_max = values.min(), values.max()

In [62]:
interp_mode="ilr"
grid_z, gr_x, gr_y = _compute_grid(barycentric_coordinates, values, interp_mode=interp_mode)

In [63]:
contour_trace, discrete_cm = _contour_trace(
    gr_x,
    gr_y,
    grid_z,
    ncontours=ncontours,
    colorscale=colorscale,
    linecolor=linecolor,
    interp_mode=interp_mode,
    coloring=coloring,
    v_min=v_min,
    v_max=v_max,
)

I can store the contour trace array somewhere, so that we can use it later to use as a prompt (maybe?) for Segment Anything Model. We can calculate the midpoint (is that even possible?) of the contour and use that as a prompt.

In [64]:
type(contour_trace[3]["a"])

numpy.ndarray

In [65]:
layout = dict(
    title=title,
    width=width,
    height=height,
    ternary=dict(
        sum=1,
        aaxis=dict(
            title=dict(text=pole_labels[0]), min=0.00, linewidth=2, ticks="outside", showgrid=False
        ),
        baxis=dict(
            title=dict(text=pole_labels[1]), min=0.00, linewidth=2, ticks="outside", showgrid=False
        ),
        caxis=dict(
            title=dict(text=pole_labels[2]), min=0.00, linewidth=2, ticks="outside", showgrid=False
        ),
        bgcolor="rgba(0,0,0,0)",
    ),
    showlegend=False,
)

In [66]:
contour_trace[0].update(
    dict(
        a=[1, 0, 0, 1],
        b=[0, 1, 0, 0],
        c=[0, 0, 1, 0],
        line=dict(color="black", width=4, shape="linear"),
    )
)

## Return value

In [67]:
go.Figure(data=contour_trace, layout=layout)

In [68]:
contour_trace[0].update(
    dict(
        a=[1, 0, 0, 1],
        b=[0, 1, 0, 0],
        c=[0, 0, 1, 0],
        line=dict(color="black", width=3, shape="linear"),
    )
)